보류
```py
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.xlabel('epoch')
# plt.ylabel('loss')
# plt.legend(['train', 'val'])
# plt.show()
# test_loss, test_acc = model.evaluate(x_test,y_test_one_hot)
# print('test_acc:', test_acc)
```

In [25]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import numpy as np
import matplotlib.pyplot as plt

In [26]:
accuracys = {}
for doc_size in [100, 400]:
    for max_feature in [10000, 20000]:
        max_features = max_feature
        (x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
        
        max_len = doc_size
        x_train = sequence.pad_sequences(x_train, maxlen=max_len)
        x_test = sequence.pad_sequences(x_test, maxlen=max_len)
        y_train_one_hot = to_categorical(y_train)
        y_test_one_hot = to_categorical(y_test)

        early_stopping = EarlyStopping(patience = 2)
        model = models.Sequential()
        model.add(layers.Embedding(max_features, 64, input_length=max_len))
        model.add(layers.SimpleRNN(32, activation='tanh', return_sequences=False))
        model.add(layers.Dense(16, activation='tanh'))
        model.add(layers.Dense(2, activation = 'softmax'))

        model.compile(optimizer=RMSprop(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])
        history=model.fit(x_train, y_train_one_hot, epochs=5,
                          batch_size=128, validation_split=0.1,
                          callbacks = [early_stopping])

        preds = model.predict(x_test)
        labels=[0,1]
        preds1 = [labels[np.argmax(probs)] for probs in preds]
        accuracys[f"{max_len},{max_feature}"] = (accuracy_score(y_test, preds1))

Epoch 1/5
176/176 [==============================] - 23s 122ms/step - loss: 0.5100 - accuracy: 0.7471 - val_loss: 0.3963 - val_accuracy: 0.8204
Epoch 2/5
176/176 [==============================] - 22s 127ms/step - loss: 0.3154 - accuracy: 0.8714 - val_loss: 0.3502 - val_accuracy: 0.8460
Epoch 3/5
176/176 [==============================] - 22s 127ms/step - loss: 0.2411 - accuracy: 0.9090 - val_loss: 0.4351 - val_accuracy: 0.8388
Epoch 4/5
782/782 [==============================] - 13s 16ms/step
Epoch 1/5
176/176 [==============================] - 24s 133ms/step - loss: 0.6087 - accuracy: 0.6528 - val_loss: 0.4966 - val_accuracy: 0.7736
Epoch 2/5
176/176 [==============================] - 22s 124ms/step - loss: 0.3586 - accuracy: 0.8530 - val_loss: 0.3938 - val_accuracy: 0.8244
Epoch 3/5
176/176 [==============================] - 21s 122ms/step - loss: 0.2440 - accuracy: 0.9055 - val_loss: 0.3591 - val_accuracy: 0.8428
Epoch 4/5
176/176 [==============================] - 23s 131ms/step -

In [30]:
accuracys

for key, accu in zip(accuracys, accuracys.values()):
    print(f"문장 길이, 코퍼스: {key}, 정확도 : {accu}")

문장 길이, 코퍼스: 100,10000, 정확도 : 0.84144
문장 길이, 코퍼스: 100,20000, 정확도 : 0.81892
문장 길이, 코퍼스: 400,10000, 정확도 : 0.86112
문장 길이, 코퍼스: 400,20000, 정확도 : 0.86636
